# Motif Finding Assignment

### ธนกิจ เหล่ายาวิระ  600632014

## Branch and Bound vs. Greedy Approach

การทดลองต่อไปนี้เป็นการค้นหา motif pattern จากข้อมูลที่โจทย์กำหนดให้ โดยโปรแกรมที่พัฒนาขึ้นนี้จะทำการค้นหา pattern ที่ต้องการเมื่อมีการกำหนดความยาว (L) ของ pattern ดังกล่าว

### How to use the program

การเรียกใช้งานโปรแกรมจะทำในรูปแบบของ command line argument ในลักษณะดังตัวอย่างต่อไปนี้

    # python a1-motif-600632014.py motif-test_sequences.fasta b 10

โดยโปรแกรมจะรับค่า argument จำนวน 3 ตัว ได้แก่
- motif-test_sequences.fasta คือ fasta ไฟล์ที่เก็บข้อมูล sequence ทั้งหมดจำนวน t ลำดับโดยแต่ละ sequence จะยาวเท่าๆกันคือ n BPs
- b คือวิธีการหา motif มี 2 วิธีคือ b = branch and bound และ g = greedy 
- 10 คือความยาวของ motif pattern ที่ต้องการค้นหา

### Important Functions

ในโปรแกรมจะประกอบไปด้วยฟังก์ชั่นที่สำคัญดังต่อไปนี้

### 1. score Function:

In [23]:
def score(s,DNA,l):
    iSeq=0
    DNACut=[]
    MaxChar=['A']*l
    MaxCharVal=[0]*l
    for iS in range(0,len(s)):
        DNACut.append(list(DNA[iSeq][(s[iS]):((s[iS])+l)]))
        iSeq += 1
    #-- cal score
    for j in range(0,l):
        cA = 0
        cT = 0
        cG = 0
        cC = 0
        for k in range(0,len(DNACut)):
            #-- count char
            if (DNACut[k][j] == 'A'):
                cA += 1
                if (cA > MaxCharVal[j]):
                    MaxCharVal[j] = cA
                    MaxChar[j] = 'A'
            elif (DNACut[k][j] == 'T'):
                cT += 1
                if (cT > MaxCharVal[j]):
                    MaxCharVal[j] = cT
                    MaxChar[j] = 'T'
            elif (DNACut[k][j] == 'G'):
                cG += 1
                if (cG > MaxCharVal[j]):
                    MaxCharVal[j] = cG
                    MaxChar[j] = 'G'
            else:
                cC += 1
                if (cC > MaxCharVal[j]):
                    MaxCharVal[j] = cC
                    MaxChar[j] = 'C'
    ResultScore = sum(MaxCharVal)
    return ResultScore,MaxChar

เป็นฟังก์ชั่นที่ใช้ในการหาค่าของ score โดยจะป้อนค่าพารามิเตอร์ได้แก่ 
1. s คือค่าของ starting position (เริ่มจาก 0) เช่น [0,0,0,0]
2. DNA คือค่าของ sequence ทั้งหมด เช่น ['AGTCG','GGTGG','CGTTT','AGTCC']
3. l คือค่าความยาวของ pattern ที่ต้องการหา

output จะมี 2 ค่า คือ ค่า score และ consensus string

ตัวอย่างการใช้งาน

In [24]:
score,consensus = score([0,0,0,0],['AGTCG','GGTGG','CGTTT','AGTCC'],3)
print("score is :",score)
print("consensus string is :",consensus)

score is : 10
consensus string is : ['A', 'G', 'T']


### 2. partialScore Function:

In [25]:
def partialScore(s,i,DNA,l):
    iSeq=0
    DNACut=[]
    MaxChar=['A']*l
    MaxCharVal=[0]*l
    for iS in range(0,i):
        DNACut.append(list(DNA[iSeq][(s[iS]):((s[iS])+l)]))
        iSeq += 1
    #-- cal score
    for j in range(0,l):
        cA = 0
        cT = 0
        cG = 0
        cC = 0
        for k in range(0,len(DNACut)):
            #-- count char
            if (DNACut[k][j] == 'A'):
                cA += 1
                if (cA > MaxCharVal[j]):
                    MaxCharVal[j] = cA
                    MaxChar[j] = 'A'
            elif (DNACut[k][j] == 'T'):
                cT += 1
                if (cT > MaxCharVal[j]):
                    MaxCharVal[j] = cT
                    MaxChar[j] = 'T'
            elif (DNACut[k][j] == 'G'):
                cG += 1
                if (cG > MaxCharVal[j]):
                    MaxCharVal[j] = cG
                    MaxChar[j] = 'G'
            else:
                cC += 1
                if (cC > MaxCharVal[j]):
                    MaxCharVal[j] = cC
                    MaxChar[j] = 'C'
    ResultScore = sum(MaxCharVal)
    return ResultScore,MaxChar

เป็นฟังก์ชั่นที่ใช้ในการหาค่าของ partial score โดยจะป้อนค่าพารามิเตอร์ได้แก่ 
1. s คือค่าของ starting position (เริ่มจาก 0) เช่น [0,0,0,0]
2. i คือค่าของจำนวน sequence ที่เราทราบค่า เช่น 2 
2. DNA คือค่าของ sequence ทั้งหมด เช่น ['AGTCG','GGTGG','CGTTT','AGTCC']
3. l คือค่าความยาวของ pattern ที่ต้องการหา

output จะมี 2 ค่า คือ ค่า partial score และ consensus string

ตัวอย่างการใช้งาน

In [26]:
score,consensus = partialScore([0,0,0,0],2,['AGTCG','GGTGG','CGTTT','AGTCC'],3)
print("partialScore is :",score)
print("consensus string is :",consensus)

partialScore is : 5
consensus string is : ['A', 'G', 'T']


## Branch and Bound algorithm for Motif Finding

### หลักการประยุกต์ใช้ Branch and Bound กับ Motif Finding

มีหลักการโดยสรุปดังต่อไปนี้
branch and bound motif search จะมีวิธีคิดโดยใช้ tree มาคำนวณหา score ที่ดีที่สุด โดยข้ามบาง node ของ tree ที่ซ้ำกัน เพื่อลดระยะเวลาการคำนวณลง

### Branch and Bound Code:

In [35]:
def branchAndBoundMotifSearch(DNA,t,n,l):
    progress = "."
    countLoop = 0
    s = [0]*t
    scoreNow = 0
    BestScore = 0
    BestMotif = [0]*t
    i = 1
    while (i > 0):
        if (i < t):
            OptimisticScore = partialScore(s,i,DNA,l)[0]+(t-i)*l
            if (OptimisticScore < BestScore):
                s,i = byPass(s,i,t,(n - l + 1))
            else:
                s,i = nextVertex(s,i,t,(n - l + 1))
        else:
            scoreNow = score(s,DNA,l)[0]
            if (scoreNow > BestScore):
                BestScore = scoreNow
                BestMotif = s.copy()
            s,i = nextVertex(s,i,t,(n - l + 1))
        countLoop += 1
        #-- print progress
        if (countLoop%100000 == 0):
            os.system('cls')
            print("==================")
            print("Loading ",progress)
            print("==================")
            if (len(progress) == 10):
                progress = "."
            else:
                progress += "."
                
    Consensus = score(BestMotif,DNA,l)[1]
    ResultBestMotif = [x+1 for x in BestMotif]
    return countLoop,ResultBestMotif,BestScore,Consensus

### Branch and Bound Result

ผลการทดลองโดยใช้ Branch and Bround algorithm

In [36]:
%run a1-motif-600632014.py motif-test_sequences.fasta b 10

Number of visiting nodes : 961044
BestMotif : [24, 1, 43, 18, 20]
BestScore : 42
Consensus ['G', 'T', 'A', 'G', 'A', 'T', 'G', 'G', 'G', 'G']


## Greedy algorithm


### หลักการประยุกต์ใช้ Greedy กับ Motif Finding

มีหลักการโดยสรุปดังต่อไปนี้ greedy จะคำนวณหา 2 starting position ที่ดีที่สุดก่อน แล้วจึงคำนวณหา starting position ตัวต่อไปจนครบ โดยไม่สนใจว่า 2 ตัวแรกจะเป็นตัวที่ดีที่สุดจริงๆหรือไม่  


### Greedy Code:


In [ ]:
# This is Motif Finding using Greedy approach
def greedyMotifSearch(DNA,t,n,l):
    countLoop = 0
    BestMotif = [0]*t
    s = [0]*t
    for s[0] in range(0,(n-l+1)):
        for s[1] in range(0,(n-l+1)):
            if (partialScore(s,2,DNA,l)[0] > partialScore(BestMotif,2,DNA,l)[0]):
                BestMotif[0] = s[0]
                BestMotif[1] = s[1]
            countLoop += 1
    s[0] = BestMotif[0]
    s[1] = BestMotif[1]
    for i in range(2,t):
        for s[i] in range(0,(n-l+1)):
            if (partialScore(s,i+1,DNA,l)[0] > partialScore(BestMotif,i+1,DNA,l)[0]):
                BestMotif[i] = s[i]
            countLoop += 1
        s[i] = BestMotif[i]
    #print(countLoop)
    BestScore = score(BestMotif,DNA,l)[0]
    Consensus = score(BestMotif,DNA,l)[1]
    ResultBestMotif = [x+1 for x in BestMotif]   
    
    return countLoop,ResultBestMotif,BestScore,Consensus




### Greedy Result

ผลการทดลองโดยใช้ Greedy algorithm

In [37]:
%run a1-motif-600632014.py motif-test_sequences.fasta g 10

Number of visiting nodes : 2754
BestMotif : [17, 39, 21, 5, 10]
BestScore : 36
Consensus ['A', 'A', 'T', 'C', 'A', 'G', 'C', 'T', 'T', 'A']


## Summary and Discussions

### 1. เปรียบเทียบประสิทธิภาพ (performance comparison)
ในแง่ความเร็วนั้น Greedy จะเร็วกว่า Branch and Bround มาก เนื่องจาก greedy จะคำนวณแค่ 2 ตัวแรกก่อนจึงตัดการเข้า node อื่นๆไปได้มาก 

### 2. เปรียบเทียบความถูกต้องของผลลัพธ์ (result comparison)
ในเรื่องความถูกต้องนั้น Branch and Bround ดีกว่า Greedy เนื่องจากผลลัพธ์ของ Branch and Bround นั้นคือผลลัพธ์ที่ดีที่สุด 
แต่ผลลัพธ์ของ Greedy นั้นอาจจะไม่ใช่ผลลัพธ์ที่ดีที่สุด

### 3. แนวทางการปรับปรุงด้านประสิทธิภาพ และความถูกต้อง (improvement)
อยากปรับปรุง ในส่วนของการคำนวณ เพราะผมใช้ index ตัวแรกเป็น 0 แต่ถ้าปรับเป็น psudocode ควรเป็น 1 เพราะคนอ่านจะเข้าใจง่ายกว่า